In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/329T/Project/

[Errno 2] No such file or directory: '/content/drive/Shareddrives/329T/Project/'
/content


In [ ]:
%cd /content/drive/MyDrive/329T Project/

/content/drive/MyDrive/329T Project


In [ ]:
# Install packages 
!pip3 install transformers 
!pip3 install torch

In [ ]:
import warnings
import os

import torch
import torch.nn as nn
import json
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from torch.nn.functional import cosine_similarity
from transformers import BertConfig, BertTokenizer, AutoConfig, AutoTokenizer, DistilBertConfig, DistilBertTokenizer
from transformers import BertForSequenceClassification, BertForMaskedLM, AutoModelForSequenceClassification, DistilBertForMaskedLM
import copy
import argparse
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import re
import copy

In [ ]:
import sys
import importlib
import subprocess
from pathlib import Path

def install_if_not_installed(packages):
  """Install the given packages if they are not already installed."""

  for package in packages:
    if isinstance(package, tuple):
      package_name, package_package = package
    else:
      package_name = package
      package_package = package

    print(f"{package_name} ... ", end='')

    try:
      importlib.import_module(package_name)
      print("already installed")

    except:
      print(f"installing from {package_package}")
      subprocess.check_call(
          [sys.executable, "-m", "pip", "install", package_package]
      )

install_if_not_installed(
    [("trulens", "git+https://github.com/truera/trulens.git@piotrm/vis/output-detect"),
     "pandas",
     "numpy",
     "domonic"]
)

trulens ... already installed
pandas ... already installed
numpy ... already installed
domonic ... already installed


In [ ]:
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.models import get_model_wrapper
from trulens.nn.attribution import Cut, OutputCut
from trulens.nn.quantities import MaxClassQoI, ClassQoI, ComparativeQoI
from torch.nn.functional import cosine_similarity
from trulens.visualizations import NLP

##### Args

In [ ]:
device = 'cuda'
data_path = "tweet_sentiment_pairs.csv"
mlm_path = "distilbert-base-uncased"
tgt_path = "distilbert-base-uncased-finetuned-sst-2-english"
output_dir = "perturbations.csv"
use_sim_mat = 0 
start = 0
end = 100
num_label = 2 
use_bpe = 1
k = 5
threshold_pred_score = 0

##### Initialize models

In [ ]:
tokenizer_mlm = DistilBertTokenizer.from_pretrained(mlm_path, do_lower_case=True)
tokenizer_tgt = AutoTokenizer.from_pretrained(tgt_path, do_lower_case=True)

config_atk = DistilBertConfig.from_pretrained(mlm_path)
mlm_model = DistilBertForMaskedLM.from_pretrained(mlm_path, config=config_atk)
mlm_model.to(device)

config_tgt = AutoConfig.from_pretrained(tgt_path, num_labels=num_label)
tgt_model = AutoModelForSequenceClassification.from_pretrained(tgt_path, config=config_tgt)
tgt_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# For pytorch, input_shape is required... I just used the tokenizer max_length here
wrapped_model = get_model_wrapper(tgt_model, input_shape=(None, 128), device=device)

INFO: Detected pytorch backend for <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>.
INFO: Using backend Backend.PYTORCH.
INFO: If this seems incorrect, you can force the correct backend by passing the `backend` parameter directly into your get_model_wrapper call.


### Attack code from BERT-Attack

In [ ]:
device = 'cuda'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)

filter_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost',
                'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another',
                'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as',
                'at', 'back', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides',
                'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', "couldn't", 'd', 'didn',
                "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 'either', 'else', 'elsewhere',
                'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'first', 'for',
                'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence',
                'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his',
                'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's",
                'its', 'itself', 'just', 'latter', 'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn',
                "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 'must', 'mustn', "mustn't", 'my', 'myself',
                'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 'no', 'nobody', 'none',
                'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only',
                'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per',
                'please', 's', 'same', 'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow',
                'something', 'sometime', 'somewhere', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs',
                'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein',
                'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 'thru', 'thus', 'to', 'too',
                'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', "wasn't",
                'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where',
                'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while',
                'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'with', 'within', 'without', 'won',
                "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you', "you'd", "you'll", "you're", "you've",
                'your', 'yours', 'yourself', 'yourselves', '-', ',', '.', ':', ';', '(', ')', '"', "'", '|', '...']
filter_words = set(filter_words)


def get_sim_embed(embed_path, sim_path):
    id2word = {}
    word2id = {}

    with open(embed_path, 'r', encoding='utf-8') as ifile:
        for line in ifile:
            word = line.split()[0]
            if word not in id2word:
                id2word[len(id2word)] = word
                word2id[word] = len(id2word) - 1

    cos_sim = np.load(sim_path)
    return cos_sim, word2id, id2word



class Feature(object):
    def __init__(self, seq_a, label, example_index):
        self.example_index = example_index
        self.label = label
        self.pred = -1
        self.seq = seq_a
        self.final_adverse = seq_a
        self.query = 0
        self.change = 0
        self.success = 0
        self.sim = 0.0
        self.metrics = None
        self.changes = []
        self.attributions = None


def _tokenize(seq, tokenizer):
   seq = seq.replace('\n', '').lower()
   words = seq.split(' ')
 
   sub_words = []
   keys = []
   index = 0
   for word in words:
       sub = tokenizer.tokenize(word)
       sub_words += sub
       keys.append([index, index + len(sub)])
       index += len(sub)
 
   return words, sub_words, keys
 
 
def _get_masked(words):
   len_text = len(words)
   masked_words = []
   for i in range(len_text - 1):
       masked_words.append(words[0:i] + ['[UNK]'] + words[i + 1:])
   # list of words
   return masked_words
 
 
def get_important_scores(words, tgt_model, orig_prob, orig_label, orig_probs, tokenizer_tgt, batch_size, max_length):
   masked_words = _get_masked(words)
   texts = [' '.join(words) for words in masked_words]  # list of text of masked words
   all_input_ids = []
   all_masks = []
   all_segs = []
   for text in texts:
       inputs = tokenizer_tgt.encode_plus(text, None, add_special_tokens=True, max_length=max_length, return_token_type_ids=True, truncation=True)
       input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
       attention_mask = [1] * len(input_ids)
       padding_length = max_length - len(input_ids)
       input_ids = input_ids + (padding_length * [0])
       token_type_ids = token_type_ids + (padding_length * [0])
       attention_mask = attention_mask + (padding_length * [0])
       all_input_ids.append(input_ids)
       all_masks.append(attention_mask)
       all_segs.append(token_type_ids)
   seqs = torch.tensor(all_input_ids, dtype=torch.long)
   masks = torch.tensor(all_masks, dtype=torch.long)
   segs = torch.tensor(all_segs, dtype=torch.long)
   seqs = seqs.to(device)
 
   eval_data = TensorDataset(seqs)
   # Run prediction for full data
   eval_sampler = SequentialSampler(eval_data)
   eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=batch_size)
   leave_1_probs = []
   for batch in eval_dataloader:
       masked_input, = batch
       bs = masked_input.size(0)
 
       leave_1_prob_batch = tgt_model(masked_input)[0]  # B num-label
       leave_1_probs.append(leave_1_prob_batch)
   leave_1_probs = torch.cat(leave_1_probs, dim=0)  # words, num-label
   leave_1_probs = torch.softmax(leave_1_probs, -1)  #
   leave_1_probs_argmax = torch.argmax(leave_1_probs, dim=-1)
   import_scores = (orig_prob
                    - leave_1_probs[:, orig_label]
                    +
                    (leave_1_probs_argmax != orig_label).float()
                    * (leave_1_probs.max(dim=-1)[0] - torch.index_select(orig_probs, 0, leave_1_probs_argmax))
                    ).data.cpu().numpy()
 
   return import_scores
 
 
def get_substitues(substitutes, tokenizer, mlm_model, use_bpe, substitutes_score=None, threshold=3.0):
   # substitues L,k
   # from this matrix to recover a word
   words = []
   sub_len, k = substitutes.size()  # sub-len, k
 
   if sub_len == 0:
       return words
 
   elif sub_len == 1:
       for (i,j) in zip(substitutes[0], substitutes_score[0]):
           if threshold != 0 and j < threshold:
               break
           words.append(tokenizer._convert_id_to_token(int(i)))
   return words
 
 
def get_bpe_substitues(substitutes, tokenizer, mlm_model):
   # substitutes L, k
 
   substitutes = substitutes[0:5, 0:4] # maximum BPE candidates # 0:12
 
   # find all possible candidates
 
   all_substitutes = []
   for i in range(substitutes.size(0)):
       if len(all_substitutes) == 0:
           lev_i = substitutes[i]
           all_substitutes = [[int(c)] for c in lev_i]
       else:
           lev_i = []
           for all_sub in all_substitutes:
               for j in substitutes[i]:
                   lev_i.append(all_sub + [int(j)])
           all_substitutes = lev_i
 
   # all substitutes  list of list of token-id (all candidates)
   c_loss = nn.CrossEntropyLoss(reduction='none')
   word_list = []
   # all_substitutes = all_substitutes[:24]
   all_substitutes = torch.tensor(all_substitutes) # [ N, L ]
   all_substitutes = all_substitutes[:24].to(device)
   N, L = all_substitutes.size()
   word_predictions = mlm_model(all_substitutes)[0] # N L vocab-size
   ppl = c_loss(word_predictions.view(N*L, -1), all_substitutes.view(-1)) # [ N*L ]
   ppl = torch.exp(torch.mean(ppl.view(N, L), dim=-1)) # N
   _, word_list = torch.sort(ppl)
   word_list = [all_substitutes[i] for i in word_list]
   final_words = []
   for word in word_list:
       tokens = [tokenizer._convert_id_to_token(int(i)) for i in word]
       text = tokenizer.convert_tokens_to_string(tokens)
       final_words.append(text)
   return final_words
 
def num_tokens(seq):
 inputs = tokenizer_tgt.encode_plus(seq, None, add_special_tokens=True, max_length=128, return_token_type_ids=True, truncation=True)
 return torch.tensor(inputs["input_ids"]).size(0)
 
def attack(feature, tgt_model, mlm_model, tokenizer_tgt, tokenizer_mlm, k, batch_size, max_length=128, cos_mat=None, w2i={}, i2w={}, use_bpe=1, threshold_pred_score=0.3):
   features = []
 
   # Get words, subwords, and the indices corresponding to that word in the tokenized sequence
   words, sub_words, keys = _tokenize(feature.seq, tokenizer_tgt)
 
   # Get original label (orig_label) and current probability (current_prob)
   # Feed in original sequence (feature.seq) into target model
   inputs = tokenizer_tgt.encode_plus(feature.seq, None, add_special_tokens=True, max_length=max_length, return_token_type_ids=True, truncation=True)
   input_ids, token_type_ids = torch.tensor(inputs["input_ids"]), torch.tensor(inputs["token_type_ids"])
   attention_mask = torch.tensor([1] * len(input_ids))
   seq_len = input_ids.size(0)
   num_tokens_original = seq_len
   orig_probs = tgt_model(input_ids.unsqueeze(0).to(device),
                          attention_mask=attention_mask.unsqueeze(0).to(device),
                          )[0].squeeze()
   orig_probs = torch.softmax(orig_probs, -1)
   orig_label = torch.argmax(orig_probs)
   current_prob = orig_probs.max()
 
   # If the original label is already different from the label in the feature, it means the model is wrong already
   if orig_label != feature.label:
       feature.success = 3
       features.append(copy.deepcopy(feature))
 
   # Feed subwords into MLM model
   sub_words = ['[CLS]'] + sub_words[:max_length - 2] + ['[SEP]']
   input_ids_ = torch.tensor([tokenizer_mlm.convert_tokens_to_ids(sub_words)])
   # For each subword in the sequence, outputs a prediction (vocab size vector)
   word_predictions = mlm_model(input_ids_.to(device))[0].squeeze()  # seq-len(sub) vocab
   # word_pred_scores_all is the top k values, word_predictions is their corresponding indices
   word_pred_scores_all, word_predictions = torch.topk(word_predictions, k, -1)  # seq-len k
   # Top k predictions (indices) for subwords (other than CLS and SEP)
   word_predictions = word_predictions[1:len(sub_words) + 1, :]
   # Top k predictions (probabilities) for subwords (other than CLS and SEP)
   word_pred_scores_all = word_pred_scores_all[1:len(sub_words) + 1, :]
   # Get importance scores for each word
   important_scores = get_important_scores(words, tgt_model, current_prob, orig_label, orig_probs,
                                           tokenizer_tgt, batch_size, max_length)
   # Store number of words??
   feature.query += int(len(words))
   # Get indices of words in decreasing importance score order
   list_of_index = sorted(enumerate(important_scores), key=lambda x: x[1], reverse=True)
   final_words = copy.deepcopy(words)
 
   # Iterate through words in decreasing importance score order
   for top_index in list_of_index:
       # If number of changes exceeds 40% of the whole sequence, return
       if feature.change > int(0.4 * (len(words))): # 0.4 is the perturbation percentage
           feature.success = 1  # exceed
           features.append(copy.deepcopy(feature))
           return features
 
       # tgt_word is the word to replace
       tgt_word = words[top_index[0]]
       if tgt_word in filter_words:
           continue
       if keys[top_index[0]][0] > max_length - 2:
           continue
 
       # For each of L tokens in the word, top k predictions
       substitutes = word_predictions[keys[top_index[0]][0]:keys[top_index[0]][1]]  # L, k
       word_pred_scores = word_pred_scores_all[keys[top_index[0]][0]:keys[top_index[0]][1]]
       # Getting most likely substitutes based on bpe
       substitutes = get_substitues(substitutes, tokenizer_mlm, mlm_model, use_bpe, word_pred_scores, threshold_pred_score)
 
 
       most_gap = 0.0
       candidate = None
       for substitute_ in substitutes:
           substitute = substitute_
 
           if substitute == tgt_word:
               continue  # filter out original word
           if '##' in substitute:
               continue  # filter out sub-word
 
           if substitute in filter_words:
               continue
           if substitute in w2i and tgt_word in w2i:
               if cos_mat[w2i[substitute]][w2i[tgt_word]] < 0.4:
                   continue
 
 
           temp_replace = final_words
           # Replace word with with substitute
           temp_replace[top_index[0]] = substitute
           # Convert back to text
           temp_text = tokenizer_tgt.convert_tokens_to_string(temp_replace)
           # Tokenize with target model tokenizer
           inputs = tokenizer_tgt.encode_plus(temp_text, None, add_special_tokens=True, max_length=max_length, )
           # Feed into target model
           input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to(device)
           seq_len = input_ids.size(1)
           temp_prob = tgt_model(input_ids)[0].squeeze()
           # Increment query
           feature.query += 1
           # Get output label
           temp_prob = torch.softmax(temp_prob, -1)
           temp_label = torch.argmax(temp_prob)
 
           # Criteria
           original_attribution, perturbed_attribution = calculate_IGs(IG_calc_attribution, feature.seq, temp_text)
           # Filter out different shape ones! # if original_attribution.shape == perturbed_attribution.shape:
           similarity = cosine_similarity(original_attribution.flatten(), perturbed_attribution.flatten(), dim=0)
 
           if temp_label != orig_label:
               # If succeeded, increment change
               feature.change += 1
               # Make the change
               final_words[top_index[0]] = substitute
               # Append the actual changes to changes
               feature.changes.append([keys[top_index[0]][0], substitute, tgt_word])
               # Store the adversarial example text in final_adverse
               feature.final_adverse = temp_text
               feature.success = 4
               features.append(copy.deepcopy(feature))
           else:
 
               label_prob = temp_prob[orig_label]
               # Look at gap between the probabilities
               gap = current_prob - label_prob
               # Store the example with the most gap as candidate
               if gap > most_gap:
                   most_gap = gap
                   candidate = substitute
               feature.final_adverse = temp_text
               features.append(copy.deepcopy(feature))
 
       if most_gap > 0:
           feature.change += 1
           feature.changes.append([keys[top_index[0]][0], candidate, tgt_word])
           current_prob = current_prob - most_gap
           final_words[top_index[0]] = candidate
 
   feature.final_adverse = (tokenizer_tgt.convert_tokens_to_string(final_words))
   feature.success = 2
   features.append(copy.deepcopy(feature))
   # Filter out adversarial examples with changed number of tokens
   features = [feature for feature in features if num_tokens(feature.final_adverse) == num_tokens_original]
   return features


##### Load data

In [ ]:
def preprocess_df(df):
  df['text'] = df['text'].apply(lambda x: re.sub(r'http[^\s]*', '', x))
  df['text'] = df['text'].apply(lambda x: re.sub('@[^\s]+','', x))
  df['text'] = df['text'].apply(lambda x: re.sub(r'#', '', x))
  df['text'] = df['text'].apply(lambda x: re.sub(r'-', '', x))
  df['text'] = df['text'].apply(lambda x: x.strip())
  return df

In [ ]:
def get_data_cls(data_path):
    df = pd.read_csv(data_path)
    df = preprocess_df(df)
    features = []
    seqs = list(df['text'])
    labels = list(df['sentiment'])
    features = list(zip(seqs, labels))
    return features 
    
features = get_data_cls(data_path)
print(features[:5])

[("Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 0), ("is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0), ('I dived many times for the ball. Managed to save 50%  The rest go out of bounds', 0), ('my whole body feels itchy and like its on fire', 0), ("no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.", 0)]


##### Generate adversarial examples

In [ ]:
if use_sim_mat == 1:
    cos_mat, w2i, i2w = get_sim_embed('data_defense/counter-fitted-vectors.txt', 'data_defense/cos_sim_counter_fitting.npy')
else:
    cos_mat, w2i, i2w = None, {}, {}

features_output = []

# old_time = time.time()

start = 0 
end = 50
outputs = []

for index, feature in enumerate(tqdm(features[start:end])):
    seq_a, label = feature
    feat = Feature(seq_a, label, index + start)

    # print('\r number {:d} '.format(index) + tgt_path, end='')
    feature = attack(feat, tgt_model, mlm_model, tokenizer_tgt, tokenizer_mlm, k, batch_size=32, max_length=128,
                  cos_mat=cos_mat, w2i=w2i, i2w=i2w, use_bpe=use_bpe,threshold_pred_score=threshold_pred_score)

    if feature:
        outputs.append({'label': feature.label,
                        'prediction': feature.pred,
                        'success': feature.success,
                        'change': feature.change,
                        'num_word': len(feature.seq.split(' ')),
                        'query': feature.query,
                        'changes': feature.changes,
                        'seq_a': feature.seq,
                        'adv': feature.final_adverse,
                        })
        
    if index % 100 == 0:
      df = pd.DataFrame(outputs)
      df.to_csv('adversarial_examples_cosine.csv')

100%|██████████| 5000/5000 [1:06:26<00:00,  1.25it/s]


##### Save generated examples to csv

In [ ]:
output_dir = "perturbations.csv"

def dump_features(features):
    outputs = []

    for feature in features:
        if feature:
            outputs.append({'label': feature.label,
                            'prediction': feature.pred,
                            'success': feature.success,
                            'change': feature.change,
                            'num_word': len(feature.seq.split(' ')),
                            'query': feature.query,
                            'changes': feature.changes,
                            'seq_a': feature.seq,
                            'adv': feature.final_adverse,
                            })

    df = pd.DataFrame(outputs)
    df.to_csv(output_dir)
    print('finished dump')
    return df

df = dump_features(features_output)

finished dump
